Purpose is to figure out why the raw sequences are messed up.

In [35]:
#!/usr/bin/env python
#map_motif.py
#Ciera Martinez

import re
import pandas as pd
import numpy as np
import os, sys
from Bio import motifs
from Bio import SeqIO 
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC, generic_dna, generic_protein
from collections import defaultdict

In [6]:
## read in alignment and motif 
alignment = list(SeqIO.parse("../../montium_data/4_alignments/align_outlier_rm_with_length_VT11157.fa", "fasta"))
motif = motifs.read(open("../data/Dmel_pwms/bcd_FlyReg.fm"), "pfm")

In [8]:
## Used later

raw_sequences = []
for record in alignment:
    raw_sequences.append(SeqRecord(record.seq.ungap("-"), id = record.id))

## make raw sequences all IUPAC.IUPACUnambiguousDNA()
raw_sequences_2 = []
for seq in raw_sequences:
    raw_sequences_2.append(Seq(str(seq.seq), IUPAC.IUPACUnambiguousDNA()))


In [68]:
#####################
## Motifs
## see http://biopython.readthedocs.io/en/latest/Tutorial/chapter_motifs.html#position-specific-scoring-matrices
#####################

pwm = motif.counts.normalize(pseudocounts=0.0) # Doesn't change from pwm
print(pwm)
pssm = pwm.log_odds()

motif_length = len(motif) #for later retrival of nucleotide sequence

######################
## Searching for Motifs in Sequences
######################

## Returns a list of arrays with a score for each position
## This give the score for each position
## If you print the length you get the length of the sequence minus TFBS length. 

pssm_list = [ ]
for seq in raw_sequences_2:
    pssm_list.append(pssm.calculate(seq))

## At this point there is a score for each position - length of motif
print(len(raw_sequences_2[6]))
print(len(pssm_list[6]))
print(pssm_list[6])

########################## 
## Automatic Calculation of threshold
##########################

## Ideal to find something that automatically calculates, as
## opposed to having human choosing threshold

## Approximate calculation of appropriate thresholds for motif finding 
## Patser Threshold
## It selects such a threshold that the log(fpr)=-ic(M) 
## note: the actual patser software uses natural logarithms instead of log_2, so the numbers 
## are not directly comparable. 

## distribution = pssm.distribution(background=motif.background, precision=10**4)
## patser_threshold = distribution.threshold_patser() #for use later

## for sys.out
## print("Patser Threshold is %5.3f" % patser_threshold) # Calculates Paster threshold. 


        0      1      2      3
A:   0.09   0.73   0.40   0.32
C:   0.08   0.15   0.20   0.63
G:   0.41   0.08   0.20   0.03
T:   0.42   0.04   0.20   0.02

3077
3074
[-6.1402273 -5.1402273 -0.930774  ... -4.1402273 -6.2016277 -5.1402273]


In [97]:
###################################
## Searching for motif in all raw_sequences
#################################

raw_id = []
for seq in raw_sequences:
    raw_id.append(seq.id)

record_length = []
for record in raw_sequences_2:
    record_length.append(len(record))
 
print(len(raw_sequences_2[6])) 
# print(raw_sequences_2[6])
print(record_length)
print(raw_id)

test = pssm.calculate(raw_sequences_2[6])
test = pd.DataFrame(test)
#print(test)

## The problem is that calculate and search are different and without a threshold, search acts weird.
## This gives you an array of positions. 
score = []
for seq in raw_sequences_2:
    counter = 0
    score.append(pssm.calculate(raw_sequences_2[counter]))
    counter + 1 
    
print(len(score))
    
# for position, score in pssm.search(raw_sequences_2[6]):
#         positions = {'species': raw_id[i], 'score':score, 'position':position, 'seq_len': record_length[i] }
#         position_list.append(positions)

# #print(pd.DataFrame(position_list))
# df = pd.DataFrame(position_list)
# print(df.shape)
# print(df.sort_values(by=['species','position'], ascending=[True, False]))

# print((3077 * 24/2))
        
## position_list = []
## for i in range(len(raw_id)):
##    for position, score in pssm.search(raw_sequences_2[i]):
##        positions = {'species': raw_id[i], 'score':score, 'position':position, 'seq_len': record_length[i] }
##        position_list.append(positions)

## position_DF = pd.DataFrame(position_list)
## position_DF = position_DF.sort_values(by=['species','position'], ascending=[True, False])
## print(position_DF)



3077
[3291, 3518, 3054, 3670, 2926, 3695, 3077, 3127, 2823, 2824, 2530, 2621, 2591, 2900, 1680, 3314, 3095, 3306, 2986, 3006, 3292, 2491, 2840, 2631]
['VT11157|1|MEMB002B|-|3291', 'VT11157|1|MEMB004A|+|3518', 'VT11157|1|MEMB007B|-|3054', 'VT11157|1|MEMB002C|-|3670', 'VT11157|1|MEMB003C|-|2926', 'VT11157|1|MEMB002A|+|3695', 'VT11157|1|MEMB002F|-|3077', 'VT11157|1|MEMB007D|-|3127', 'VT11157|1|MEMB007C|+|2823', 'VT11157|1|MEMB003B|+|2824', 'VT11157|1|MEMB006B|+|2530', 'VT11157|1|MEMB003F|+|2621', 'VT11157|1|MEMB002E|+|2591', 'VT11157|1|MEMB002D|+|2900', 'VT11157|1|MEMB005D|-|1680', 'VT11157|1|MEMB006A|-|3314', 'VT11157|1|dkik|-|3095', 'VT11157|1|MEMB003A|-|3306', 'VT11157|1|MEMB003D|-|2986', 'VT11157|1|MEMB004E|-|3006', 'VT11157|1|MEMB004B|-|3292', 'VT11157|1|MEMB005B|-|2491', 'VT11157|1|MEMB006C|+|2840', 'VT11157|1|MEMB008C|-|2631']
24


In [112]:

raw_id = []
for seq in raw_sequences:
    raw_id.append(seq.id)

record_length = []
for record in raw_sequences_2:
    record_length.append(len(record))

position_list = []
for i in range(len(raw_id)):
    for position, score in pssm.search(raw_sequences_2[i], threshold = -50):
        positions = {'species': raw_id[i], 'score':score, 'position':position, 'seq_len': record_length[i] }
        position_list.append(positions)
        
position_DF = pd.DataFrame(position_list)
position_DF = position_DF.sort_values(by=['species','position'], ascending=[True, False])
print(position_DF)

       position     score  seq_len                    species
40270      3691 -6.201628     3695  VT11157|1|MEMB002A|+|3695
40268      3690 -4.140227     3695  VT11157|1|MEMB002A|+|3695
40266      3689 -6.140227     3695  VT11157|1|MEMB002A|+|3695
40264      3688 -5.140227     3695  VT11157|1|MEMB002A|+|3695
40262      3687 -8.403262     3695  VT11157|1|MEMB002A|+|3695
40260      3686 -1.953700     3695  VT11157|1|MEMB002A|+|3695
40258      3685 -6.555265     3695  VT11157|1|MEMB002A|+|3695
40256      3684 -2.307337     3695  VT11157|1|MEMB002A|+|3695
40254      3683 -3.140227     3695  VT11157|1|MEMB002A|+|3695
40252      3682 -3.345811     3695  VT11157|1|MEMB002A|+|3695
40250      3681 -1.193808     3695  VT11157|1|MEMB002A|+|3695
40248      3680 -1.953700     3695  VT11157|1|MEMB002A|+|3695
40246      3679  1.007825     3695  VT11157|1|MEMB002A|+|3695
40244      3678 -3.307337     3695  VT11157|1|MEMB002A|+|3695
40242      3677 -3.140227     3695  VT11157|1|MEMB002A|+|3695
40240   

In [119]:
#############################
## Add strand and pos position information as columns to position_DF
#############################

position_list_pos = []
for i, x in enumerate(position_DF['position']):
    if x < 0:
       position_list_pos.append(position_DF.loc[position_DF.index[i], 'seq_len'] + x)
    else:
       position_list_pos.append(x)

## append to position_DF
position_DF['raw_position'] = position_list_pos
    
## strand Column
strand = []
for x in position_DF['position']:
    if x < 0:
       strand.append("negative")
    else:
       strand.append("positive")
    
## append to position_DF
position_DF['strand'] = strand

## motif_found column
raw_sequences_2_list = []
for seq in raw_sequences_2:
    raw_sequences_2_list.append(str(seq))
    
position_DF = position_DF.sort_values(by=['species','raw_position'], ascending=[True, False])
position_DF.to_csv('map_motif_test.csv', sep='\t')

In [109]:
##################
## get alignment position 
#################

remap_list = []
nuc_list = ['A', 'a', 'G', 'g', 'C', 'c', 'T', 't', 'N', 'n']

for i in range(len(raw_id)):
    counter = 0
    for xInd, x in enumerate(alignment[i].seq):    
        if x in nuc_list:
            remaps = {'raw_position': counter, 'align_position':xInd, 'species':alignment[i].id}
            counter += 1
            remap_list.append(remaps)
            
remap_DF = pd.DataFrame(remap_list)
remap_DF = remap_DF.sort_values(by=['species','raw_position'], ascending=[True, True])


print(remap_DF)

       align_position  raw_position                    species
16459               0             0  VT11157|1|MEMB002A|+|3695
16460               1             1  VT11157|1|MEMB002A|+|3695
16461               2             2  VT11157|1|MEMB002A|+|3695
16462               3             3  VT11157|1|MEMB002A|+|3695
16463               4             4  VT11157|1|MEMB002A|+|3695
16464               5             5  VT11157|1|MEMB002A|+|3695
16465               6             6  VT11157|1|MEMB002A|+|3695
16466               7             7  VT11157|1|MEMB002A|+|3695
16467               8             8  VT11157|1|MEMB002A|+|3695
16468               9             9  VT11157|1|MEMB002A|+|3695
16469              10            10  VT11157|1|MEMB002A|+|3695
16470              11            11  VT11157|1|MEMB002A|+|3695
16471              12            12  VT11157|1|MEMB002A|+|3695
16472              13            13  VT11157|1|MEMB002A|+|3695
16473              14            14  VT11157|1|MEMB002A

In [110]:
## Merge both datasets
## Merge - all sites

TFBS_map_DF_all = pd.merge(position_DF, remap_DF, on=['species', 'raw_position'], how='outer')

print(TFBS_map_DF_all)

## Sort
TFBS_map_DF_all = TFBS_map_DF_all.sort_values(by=['species','align_position'], ascending=[True, True])

## Attach File Name
TFBS_map_DF_all['alignment_file'] = alignment_file_name
TFBS_map_DF_all['motif_file'] = motif_file_name

## Remove NAs
TFBS_map_DF_all = TFBS_map_DF_all.dropna()

##remove values under the threshold if there is one
print ("Output Threshold: " + str(threshold))
TFBS_map_DF_all = TFBS_map_DF_all.loc[TFBS_map_DF_all['score'] >= float(threshold)] 

## Write out Files
# print ("Success!!")
# print ("The output csv file is located at: " + os.getcwd())
# print ("Under the name: map_motif" + alignment_file_name + "-" + motif_file_name + ".csv")
TFBS_map_DF_all.to_csv('map_motif_test.csv', sep='\t', na_rep="NA")

        position     score  seq_len                    species  raw_position  \
0         3691.0 -6.201628   3695.0  VT11157|1|MEMB002A|+|3695          3691   
1           -4.0 -0.193808   3695.0  VT11157|1|MEMB002A|+|3695          3691   
2         3690.0 -4.140227   3695.0  VT11157|1|MEMB002A|+|3695          3690   
3           -5.0 -0.059410   3695.0  VT11157|1|MEMB002A|+|3695          3690   
4         3689.0 -6.140227   3695.0  VT11157|1|MEMB002A|+|3695          3689   
5           -6.0 -1.155334   3695.0  VT11157|1|MEMB002A|+|3695          3689   
6         3688.0 -5.140227   3695.0  VT11157|1|MEMB002A|+|3695          3688   
7           -7.0  0.940590   3695.0  VT11157|1|MEMB002A|+|3695          3688   
8         3687.0 -8.403262   3695.0  VT11157|1|MEMB002A|+|3695          3687   
9           -8.0 -3.155334   3695.0  VT11157|1|MEMB002A|+|3695          3687   
10        3686.0 -1.953700   3695.0  VT11157|1|MEMB002A|+|3695          3686   
11          -9.0  1.054119   3695.0  VT1

NameError: name 'alignment_file_name' is not defined